In [ ]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
!nvidia-smi

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
import os 
import shutil
import glob
!pip install opencv-python==3.4.13.47 
import cv2
!pip install opencv-contrib-python==3.4.13.47 --force-reinstall

In [ ]:
cv2.__version__

In [ ]:
%cd /content/
%rm -rf darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet/

In [ ]:
!sed -i '1s/GPU=0/GPU=1/' Makefile 
!sed -i '2s/CUDNN=0/CUDNN=1/' Makefile 
!sed -i '4s/OPENCV=0/OPENCV=1/' Makefile 
!sed -i '6s/OPENMP=0/OPENMP=1/' Makefile 
!sed -i '7s/LIBSO=0/LIBSO=1/' Makefile
!sed -i '17s/USE_CPP=0/USE_CPP=1/' Makefile

!make -j8
!chmod +x ./darknet

In [ ]:
!wget -P build/darknet/x64/ https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137 

In [ ]:
%cp cfg/yolov4-custom.cfg cfg/yolo-obj.cfg

!sed -i '7s/subdivisions=16/subdivisions=32/g' cfg/yolo-obj.cfg

!sed -i 's/width=416/width=608/g' cfg/yolo-obj.cfg
!sed -i 's/height=416/height=608/g' cfg/yolo-obj.cfg

!sed -i '10s/channels=3/channels=1/' cfg/yolo-obj.cfg
!sed -i '20s/max_batches = 500500/max_batches = 2000/' cfg/yolo-obj.cfg
!sed -i '22s/steps=400000,450000/steps=1600,1800/' cfg/yolo-obj.cfg

!sed -i '970 s@classes=80@classes=1@' cfg/yolo-obj.cfg
!sed -i '1058 s@classes=80@classes=1@' cfg/yolo-obj.cfg
!sed -i '1146 s@classes=80@classes=1@' cfg/yolo-obj.cfg

!sed -i '963 s@filters=255@filters=18@' cfg/yolo-obj.cfg
!sed -i '1051 s@filters=255@filters=18@' cfg/yolo-obj.cfg
!sed -i '1139 s@filters=255@filters=18@' cfg/yolo-obj.cfg

!sed -i '973 s@ignore_thresh = .7@ignore_thresh = .9@' cfg/yolo-obj.cfg
!sed -i '978 s@iou_normalizer=0.07@iou_normalizer=0.5@' cfg/yolo-obj.cfg
!sed -i '979 s@iou_loss=ciou@iou_loss=giou@' cfg/yolo-obj.cfg

!sed -i '1061 s@ignore_thresh = .7@ignore_thresh = .9@' cfg/yolo-obj.cfg
!sed -i '1066 s@iou_normalizer=0.07@iou_normalizer=0.5@' cfg/yolo-obj.cfg
!sed -i '1067 s@iou_loss=ciou@iou_loss=giou@' cfg/yolo-obj.cfg

!sed -i '1149 s@ignore_thresh = .7@ignore_thresh = .9@' cfg/yolo-obj.cfg
!sed -i '1155 s@iou_normalizer=0.07@iou_normalizer=0.5@' cfg/yolo-obj.cfg
!sed -i '1156 s@iou_loss=ciou@iou_loss=giou@' cfg/yolo-obj.cfg


In [ ]:
# changes check
# !cat cfg/yolo-obj.cfg

In [ ]:
%mkdir build/darknet/x64/data/obj

In [ ]:
all_classes = """cone
"""

file = """text_file = open("build/darknet/x64/data/obj/classes.txt", "w");text_file.write(all_classes);text_file.close()""" 

exec(file)

In [ ]:
all_classes = """cone
"""

file = """text_file = open("build/darknet/x64/data/obj.names", "w");text_file.write(all_classes);text_file.close()""" 

exec(file)

In [ ]:
!echo -e 'classes = 1\ntrain  = build/darknet/x64/data/train.txt\nvalid  = build/darknet/x64/data/test.txt\nnames = build/darknet/x64/data/obj.names\nbackup = /content/drive/My Drive/backup/' > build/darknet/x64/data/obj.data

In [ ]:
# check
!cat build/darknet/x64/data/obj.data
print()
!cat build/darknet/x64/data/obj.names

classes = 1
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/test.txt
names = build/darknet/x64/data/obj.names
backup = /content/drive/My Drive/backup/

cone


In [ ]:
dir = '/content/drive/My Drive/backup/'
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
%cp -a cfg/yolo-obj.cfg '/content/drive/My Drive/backup/'
%cp -a build/darknet/x64/data/obj.names '/content/drive/My Drive/backup/'
%cp build/darknet/x64/data/obj/classes.txt '/content/drive/My Drive/backup/'

In [ ]:
yolo_dir = '/content/drive/My Drive/yolov/dataset_grayscale'
if not os.path.exists(yolo_dir):
    os.makedirs(yolo_dir)

In [ ]:
# !unzip '/content/drive/My Drive/yolov/dataset_grayscale.zip' -d '/content/drive/My Drive/yolov/'

In [ ]:
!find '/content/drive/My Drive/yolov/dataset_grayscale' -name '*.jpg' -exec cp -prv '{}' 'build/darknet/x64/data/obj' ';'
!find '/content/drive/My Drive/yolov/dataset_grayscale' -name '*.txt' -exec cp -prv '{}' 'build/darknet/x64/data/obj' ';'

In [ ]:
# 7748 images
%ls build/darknet/x64/data/obj/*.jpg | wc -l

# 7748 labels + 1 classes.txt file
%ls build/darknet/x64/data/obj/*.txt | wc -l

In [ ]:
!echo -n > build/darknet/x64/data/train.txt
!echo -n > build/darknet/x64/data/test.txt
images = glob.glob('build/darknet/x64/data/obj/*.jpg')

In [ ]:
train_file = open("build/darknet/x64/data/train.txt", "w")
test_file = open("build/darknet/x64/data/test.txt", "w")

for j in images:
    if np.random.uniform(0,1) < 0.8:
        train_file.write(j + '\n')
    else:
        test_file.write(j + '\n')

train_file.close()
test_file.close()

In [ ]:
!cat -n build/darknet/x64/data/train.txt | sed '1,5!d'
print()
!cat -n build/darknet/x64/data/test.txt | sed '1,5!d'

In [ ]:
#Count number of files 
!wc -l build/darknet/x64/data/train.txt
!wc -l build/darknet/x64/data/test.txt

In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show -map | tee results.log

In [ ]:
# %mkdir video_frames 

# def split_into_frames(video_file, pathOut):
#     vid = cv2.VideoCapture(video_file)
#     count = 0
#     while vid.isOpened():
#         success, frame = vid.read()
#         if success == False:
#             break
#         print('Read a new frame', success, count)
#         if count % 600 == 0:
#             cv2.imwrite(f'{pathOut}/frame_{count}.jpg', frame)
#         count += 1

# pathOut = 'video_frames'
# split_into_frames(video_file, pathOut)